In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open(r'C:\Users\VAISHNAVI\Desktop\Chatbot\intents.json').read()
intents = json.loads(data_file)
from nltk.tokenize import word_tokenize
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            
#from
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")


47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']
Training data created
Epoch 1/200


C:\Users\VAISHN~1\AppData\Local\Temp/ipykernel_5244/2919959101.py:67: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)
C:\Users\VAISHNAVI\anaconda3\lib\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


10/10 [==============================] - 0s 3ms/step - loss: 2.1980 - accuracy: 0.1277
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1300 - accuracy: 0.1277
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0567 - accuracy: 0.2979
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 1.9931 - accuracy: 0.2553
Epoch 5/200
10/10 [==============================] - 0s 1ms/step - loss: 1.8354 - accuracy: 0.3617
Epoch 6/200
10/10 [==============================] - 0s 1ms/step - loss: 1.6945 - accuracy: 0.4681
Epoch 7/200
10/10 [==============================] - 0s 3ms/step - loss: 1.6342 - accuracy: 0.5106
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 1.4450 - accuracy: 0.5957
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 1.5059 - accuracy: 0.5532
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 1.2576 - accuracy: 0.6383
Epoch 11/200
10/10 [=

10/10 [==============================] - 0s 1ms/step - loss: 0.0575 - accuracy: 0.9787
Epoch 84/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0350 - accuracy: 1.0000
Epoch 85/200
10/10 [==============================] - 0s 954us/step - loss: 0.0383 - accuracy: 1.0000
Epoch 86/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0233 - accuracy: 1.0000
Epoch 87/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0709 - accuracy: 0.9787
Epoch 88/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0479 - accuracy: 0.9787
Epoch 89/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0293 - accuracy: 1.0000
Epoch 90/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0335 - accuracy: 1.0000
Epoch 91/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0536 - accuracy: 0.9787
Epoch 92/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0730 - accuracy: 0.9787
Epoch 93/20

10/10 [==============================] - 0s 891us/step - loss: 0.0914 - accuracy: 0.9787
Epoch 164/200
10/10 [==============================] - 0s 922us/step - loss: 0.0046 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 968us/step - loss: 0.0316 - accuracy: 1.0000
Epoch 166/200
10/10 [==============================] - 0s 993us/step - loss: 0.0204 - accuracy: 1.0000
Epoch 167/200
10/10 [==============================] - 0s 888us/step - loss: 0.0251 - accuracy: 1.0000
Epoch 168/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 169/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0471 - accuracy: 1.0000
Epoch 170/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0653 - accuracy: 0.9787
Epoch 171/200
10/10 [==============================] - 0s 681us/step - loss: 0.0193 - accuracy: 1.0000
Epoch 172/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0242 - accuracy: